## Analysis of an E-commerce Dataset Part 3

In this Portfolio task, you will continue working with the dataset you have used in portfolio 2. But the difference is that the ratings have been converted to like (with score 1) and dislike (with score 0). Your task is to train classification models such as KNN to predict whether a user like or dislike an item.  
    
The goal in this notebook is to try to build and evaluate predictive models for 'rating' from other available features - predict the value of the like (corresponding to rating 1) and dislike (corresponding to rating 

## Import Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Load dataset
df = pd.read_csv("portfolio_3.csv")
df.head()

,userId,timestamp,review,item,helpfulness,gender,category,item_id,item_price,user_city,rating
0,4081,71900,Not always McCrap,McDonald's,3,M,Restaurants & Gourmet,41,30.74,4,1
1,4081,72000,I dropped the chalupa even before he told me to,Taco Bell,4,M,Restaurants & Gourmet,74,108.30,4,0
2,4081,72000,The Wonderful World of Wendy,Wendy's,4,M,Restaurants & Gourmet,84,69.00,4,1
3,4081,100399,They actually did it,"South Park: Bigger, Longer & Uncut",3,M,Movies,68,143.11,4,1
4,4081,100399,Hey! Gimme some pie!,American Pie,3,M,Movies,6,117.89,4,0


## Explore Data and Cleaning

In [4]:
# Explore the data. Clean the data if necessary. For example, remove abnormal instanaces and replace missing values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2685 entries, 0 to 2684
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   userId       2685 non-null   int64  
 1   timestamp    2685 non-null   int64  
 2   review       2685 non-null   object 
 3   item         2685 non-null   object 
 4   helpfulness  2685 non-null   int64  
 5   gender       2685 non-null   object 
 6   category     2685 non-null   object 
 7   item_id      2685 non-null   int64  
 8   item_price   2685 non-null   float64
 9   user_city    2685 non-null   int64  
 10  rating       2685 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 230.9+ KB


## Converting Object Features to Numeric

In [5]:
# Convert object features into digit features by using an encoder
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(dtype = int)
df[["review", "item", "gender", "category"]] = enc.fit_transform(df[["review", "item", "gender", "category"]])
df.head()

,userId,timestamp,review,item,helpfulness,gender,category,item_id,item_price,user_city,rating
0,4081,71900,1618,37,3,1,8,41,30.74,4,1
1,4081,72000,1125,67,4,1,8,74,108.30,4,0
2,4081,72000,2185,77,4,1,8,84,69.00,4,1
3,4081,100399,2243,61,3,1,5,68,143.11,4,1
4,4081,100399,1033,5,3,1,5,6,117.89,4,0


## Correlation Analysis

In [6]:
# Study the correlation between these features. 
df.corr()

,userId,timestamp,review,item,helpfulness,gender,category,item_id,item_price,user_city,rating
userId,1.000000,-0.069176,0.007139,-0.005513,-0.166136,-0.058324,-0.041362,-0.005549,0.024576,-0.030031,0.066444
timestamp,-0.069176,1.000000,0.007029,-0.003543,0.014179,-0.003367,0.015009,-0.004452,0.010979,-0.014934,-0.009739
review,0.007139,0.007029,1.000000,0.163090,-0.028259,-0.037884,0.001970,0.163544,-0.041421,0.045626,-0.041756
item,-0.005513,-0.003543,0.163090,1.000000,-0.020433,0.001925,-0.045988,0.999765,-0.049885,-0.005220,0.057793
helpfulness,-0.166136,0.014179,-0.028259,-0.020433,1.000000,0.075947,-0.013408,-0.019882,0.004112,0.012086,-0.010622
gender,-0.058324,-0.003367,-0.037884,0.001925,0.075947,1.000000,0.022549,0.002370,-0.040596,-0.065638,-0.022169
category,-0.041362,0.015009,0.001970,-0.045988,-0.013408,0.022549,1.000000,-0.045268,-0.115571,0.008017,-0.142479
item_id,-0.005549,-0.004452,0.163544,0.999765,-0.019882,0.002370,-0.045268,1.000000,-0.054450,-0.005576,0.057107
item_price,0.024576,0.010979,-0.041421,-0.049885,0.004112,-0.040596,-0.115571,-0.054450,1.000000,-0.023427,0.026062
user_city,-0.030031,-0.014934,0.045626,-0.005220,0.012086,-0.065638,0.008017,-0.005576,-0.023427,1.000000,-0.034866


## Split the Dataset

In [7]:
# Split the dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['rating']), df["rating"], test_size=0.2, random_state=42)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", X_test.shape)

X_train:  (2148, 10)
X_test:  (537, 10)
y_train:  (2148,)
y_test:  (537, 10)


## Train a Logistic Regression Model

In [8]:
# Train a logistic regression model to predict 'rating' based on other features. 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [9]:
# Evaluate the accuracy of your model.
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print("Accuracy on logistical regression: ", accuracy_score(y_test, y_pred))

Accuracy on logistical regression:  0.6368715083798883


### Findings
An accuracy of 63.69% for a logistic regression model on this dataset is a promising starting point, but further analysis and model iteration would be beneficial to ensure the model's robustness and to potentially improve its performance.


In [10]:
# Use RFE to improve the model
from sklearn.feature_selection import RFE
selector = RFE(clf, n_features_to_select=3)
selector = selector.fit(X_train, y_train)
selector.ranking_

array([6, 8, 7, 1, 3, 1, 1, 2, 5, 4])

In [11]:
# Use RFE selected features to retrain the model
X_train, X_test, y_train, y_test = train_test_split(df[['item', 'gender', 'category']], df["rating"], test_size=0.2, random_state=42)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", X_test.shape)

X_train:  (2148, 3)
X_test:  (537, 3)
y_train:  (2148,)
y_test:  (537, 3)


In [12]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy on logistical regression: ", accuracy_score(y_test, y_pred))

Accuracy on logistical regression:  0.6443202979515829


### Findings
Using item, gender, categroy, has led to an improved logistic regression model for our online reviews dataset. The increase from 63.69% to 64.43% represents an uplift in accuracy, although not by much.

## Train a KNN Model

In [14]:
# Train a KNN model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [15]:
# get accuracy of KNN model
y_pred = clf.predict(X_test)
print("Accuracy on KNN with selected features: ", accuracy_score(y_test, y_pred))

Accuracy on KNN with selected features:  0.6443202979515829


### Findings
The KNN classifier's performance with selected features on the online reviews dataset stands at an accuracy of 64.43%. It matches the performance of the logistic regression model with RFE-selected features. 

## Tune the Hyper-Parameter K in KNN¶

In [17]:
# Tune the hyper-parameter K in KNN
from sklearn.model_selection import GridSearchCV
parameters = {'n_neighbors': range(1,100)}
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=3),
             param_grid={'n_neighbors': range(1, 100)})

## Results

In [18]:
clf.best_params_

{'n_neighbors': 22}

In [19]:
clf.best_score_

0.7453504634899983

### Findings
The initial Logistic Regression model achieved an accuracy of 63.69% which slightly improved to 64.43% with feature selection. The KNN model matched the Logistic Regression model's performance with an accuracy of 64.43%. Hyper-parameter tuning in KNN revealed that K=22 yielded the highest cross-validation score of 74.54%.